#### Шаг 1: Генерация "грязного" CSV

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

spark = SparkSession.builder.appName("IO_Practice").getOrCreate()

# Создаем папку для данных
os.makedirs("/home/jovyan/workspace/data/raw_sales", exist_ok=True)

# Генерируем данные
data = [
    ("2023-01-01", "iPhone", "US", 1000),
    ("2023-01-01", "Samsung", "KR", 900),
    ("2023-01-02", "Xiaomi", "CN", 400),
    ("2023-01-02", "iPhone", "US", 1000)
]
columns = ["date", "product", "country", "amount"]

df = spark.createDataFrame(data, columns)

# Пишем в CSV (эмулируем входящие данные)
# header=True - записать заголовки
df.write.format("csv").mode("overwrite").option("header", "true").save("/home/jovyan/workspace/data/raw_sales")

print("CSV файлы созданы. Проверьте папку data/raw_sales через терминал.")

CSV файлы созданы. Проверьте папку data/raw_sales через терминал.


#### Шаг 2: Правильное чтение CSV
Теперь представим, что мы начали ETL-процесс. Читаем CSV.

In [2]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# 1. Определяем схему (Best Practice)
sales_schema = StructType([
    StructField("date", StringType(), True),
    StructField("product", StringType(), True),
    StructField("country", StringType(), True),
    StructField("amount", IntegerType(), True)
])

# 2. Читаем
raw_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema(sales_schema) \
    .load("/home/jovyan/workspace/data/raw_sales")

raw_df.show()

+----------+-------+-------+------+
|      date|product|country|amount|
+----------+-------+-------+------+
|2023-01-01| iPhone|     US|  1000|
|2023-01-01|Samsung|     KR|   900|
|2023-01-02| iPhone|     US|  1000|
|2023-01-02| Xiaomi|     CN|   400|
+----------+-------+-------+------+



#### Шаг 3: Конвертация в Parquet с Партиционированием
Мы хотим сохранить данные так, чтобы потом быстро искать продажи по странам. Используем partitionBy.

In [3]:
# Пишем в Parquet, разделяя по стране
raw_df.write \
    .format("parquet") \
    .mode("overwrite") \
    .partitionBy("country") \
    .save("/home/jovyan/workspace/data/processed_sales")

print("Данные записаны в Parquet с партиционированием.")

Данные записаны в Parquet с партиционированием.


#### Шаг 4: Анализ структуры на диске (Терминал)
Выполните это прямо в ноутбуке через !:

In [6]:
!ls -la /home/jovyan/workspace/data/processed_sales

total 24
drwxr-xr-x 5 jovyan users 4096 Dec 16 12:24  .
drwxr-xr-x 4 jovyan users 4096 Dec 16 12:24  ..
drwxr-xr-x 2 jovyan users 4096 Dec 16 12:24 'country=CN'
drwxr-xr-x 2 jovyan users 4096 Dec 16 12:24 'country=KR'
drwxr-xr-x 2 jovyan users 4096 Dec 16 12:24 'country=US'
-rw-r--r-- 1 jovyan users    0 Dec 16 12:24  _SUCCESS
-rw-r--r-- 1 jovyan users    8 Dec 16 12:24  ._SUCCESS.crc
